In [51]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import pandas as pd
from pandas import DataFrame 
import json
import random

In [4]:
results = []

for i in range(1,13):
    filename = 'hypia20210810/{}.json'.format(i)
    with open(filename, 'r') as file:
        result = json.loads(file.read())['result']
        if 'items' in result:
            results.append(result['items'])
        elif 'data' in result: # applies only for page 1
            results.append(result['data']['items'])

len(results)

12

In [79]:
shortDescs = [profile['shortDescription'].split('\n') for page in results for profile in page]

sample = shortDescs[random.randrange(len(shortDescs))]

sample

['Name: Kevin Fei Sun',
 'Nationality: United States. Ethnicity: Chinese',
 'Where do you live?: New York City',
 'Languages: English, Mandarin (native); Russian, Spanish, French, Portuguese, German, Serbo-Croatian, Italian, Hindi/Urdu (B2); Turkish, Korean, Japanese, Arabic, Hebrew (B1); Shanghainese (heritage speaker).']

In [96]:
keyFixes = {
    'Where do you live': 'Where do you live?',
    'Nationality': 'Nationality or Ethnicity',
    'Ethnicity': 'Nationality or Ethnicity',
    'Name and Title': 'Name',
    'Where do you live ?': 'Where do you live?',
    'Currently living in': 'Where do you live?'
}

ignoreKeys = ['*Alternatively', 'Rare language', 'Representative of rare language', 'Language Family']

def processDescription(sd):
    output = {}
    _key = ''
    for line in sd:
        #print(line)
        value = ''
        if ': ' in line:
            key, *value = line.split(': ')
            key = key.strip()
            if key in keyFixes: key = keyFixes[key]
            if key in ignoreKeys:
                output[_key] = output[_key] + ': '.join(value)
            else:
                output[key] = ': '.join(value)
                _key = key
        elif key != '':
            output[_key] = output[_key] + ': '.join(value)
    return output
        
processDescription(sample)

{'Name': 'Kevin Fei Sun',
 'Nationality or Ethnicity': 'United States. Ethnicity: Chinese',
 'Where do you live?': 'New York City',
 'Languages': 'English, Mandarin (native); Russian, Spanish, French, Portuguese, German, Serbo-Croatian, Italian, Hindi/Urdu (B2); Turkish, Korean, Japanese, Arabic, Hebrew (B1); Shanghainese (heritage speaker).'}

In [97]:
rawData = DataFrame([processDescription(sd) for sd in shortDescs])
rawData

,Name,Role at HYPIA,Languages,Nationality or Ethnicity,Where do you live?,Reading Proficiency,Mother Tongue,Fluent (C level),Upper-Intermediate (B2 level),Lower-Intermediate (B1 level),...,Reading,Some phrases,(Her Youtube channel,Family Language,Beginner in,Natuionality or Ethnicity,Learning,Currently learning,[Fluent],[Conversant]
0,Usman W. Chohan,President,"Urdu, Brazilian Portuguese, Spanish, English, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Matias Barmat,Director of Recruitment,"Spanish (native), English (C1), Catalan (C1), ...",Argentina /,Buenos Aires,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Eduardo Teiga,Director of Outreach,"Portuguese, Spanish, Catalan, English, German,...",Portuguese,Switzerland,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hugues Pluvinage,NaN,"French (native), English (C2), Dutch (C2), It...",Belgian,Italy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carlos Yebra Lopez,NaN,"Spanish, Judeo-Spanish, Catalan, Portuguese, F...",Spanish,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Jakub Jarosz,NaN,"Polish, English, Italian, Spanish, Russian, Fr...",Polish,Poland,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,George L. O’Hara,NaN,"English (native), French, Spanish, Russian, Uk...","American, Irish","Washington, DC",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,Helmar Böhnlein\t,NaN,"German, English, French, Portuguese, Spanish, ...",German,"Vienna, Austria",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,Loana Eugenie Eleonora Kontogouri,NaN,"Romanian, English, French, German, Spanish, It...",Greek & Romanian,"Munich, Germany",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
rawData.count().sort_values(ascending=False)

Name                                                         226
Nationality or Ethnicity                                     223
Languages                                                    204
Where do you live?                                           191
Role at HYPIA                                                  3
Conversational                                                 3
Mother Tongue                                                  2
Fluent                                                         2
Conversant                                                     2
Learning                                                       2
Fluent (C level)                                               2
Also a speaker of                                              1
Basic                                                          1
Read                                                           1
Currently studying (basic level)                               1
Basic (mainly written)   